In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [2]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Model data
class RouteQuery(BaseModel):
    """Arahkan kueri pengguna ke sumber data paling relevan."""

    datasource: Literal["python_docs", "js_docs"] = Field(
        ...,
        description="""Diberikan pertanyaan pengguna, pilih sumber data mana yang akan
            paling relevan untuk menjawab pertanyaan mereka""",
    )

# LLM dengan pemanggilan fungsi
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """Anda adalah ahli dalam mengarahkan pertanyaan pengguna ke sumber data
    yang sesuai.

Berdasarkan bahasa pemrograman yang dirujuk pertanyaan, arahkan ke sumber data
    relevan."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Definisikan router
router  =  prompt  |  structured_llm

In [3]:
question = """Mengapa kode berikut tidak berfungsi:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

result.datasource

'python_docs'

In [ ]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logika di sini
        return "rantai untuk python_docs"
    else:
        ### Logika di sini
        return "rantai untuk js_docs"

full_chain = router | RunnableLambda(choose_route)